- Create a new DNN that reuses all the pre-trained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a fresh new one. 
- Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision? 
- Try caching the frozen layers, and train the model again: how much faster is it now? 
- Try again reusing just four hidden layers instead of five. Can you achieve a higher precision? 
- Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

### Get Data

In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28)

train_indxes_5to9 = y_train>=5
train_x_5to9 = x_train[train_indxes_5to9]
train_y_5to9 = y_train[train_indxes_5to9]-5


def get_random_indxs(y_train, group_value, count=10):
    train_indxs = np.arange(len(y_train), dtype=np.int32)
    group_indxs = train_indxs[y_train == group_value]
    indxs = np.random.choice(group_indxs,count)
    return indxs

def get_samples_of_each_group(y_train, count=300):
    rand_indxs = np.array([], dtype=np.int32)
    for group_val in np.unique(y_train):
        rand_indxs=np.r_[rand_indxs,get_random_indxs(y_train,group_val, count)]
        
    np.random.shuffle(rand_indxs)
    return rand_indxs

rand_indxs = get_samples_of_each_group(train_y_5to9)

train_x_5to9, val_x_5to9, train_y_5to9, val_y_5to9 = \
    train_test_split(train_x_5to9, train_y_5to9, test_size=.3)

val_x_5to9, test_x_5to9, val_y_5to9, test_y_5to9 = \
    train_test_split(val_x_5to9,val_y_5to9, test_size=.6 )

/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Verify that DNN_Classifier is not broken

In [ ]:
from imp import reload
import my_libs
reload(my_libs.dnn)
from my_libs.dnn import DNN_Classifier

classifier = DNN_Classifier(n_hidden_layers=3, n_neurons=100,n_outputs=5,
                            batch_norm_momentum=.95)

classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)

### Create DNN_Classifier_Transfer
which will transfer learning of DNN_Classifier from saved file

In [146]:
from imp import reload
import my_libs
reload(my_libs.dnn)

import tensorflow as tf
from my_libs.dnn import DNN_Classifier, get_optimizer_op, get_validation_score, \
    get_softmax_xentropy_loss
from tensorflow.train import AdamOptimizer

class DNN_Classifier_Transfer(DNN_Classifier):
    def __init__(self, checkpoint_name, reuse_layers=1):
        DNN_Classifier.__init__(self)
        self._checkpoint_name = checkpoint_name
        self._batch_norm_update_ops = None
        
    
    def _initialize_session_and_graph(self):
        DNN_Classifier._restore_graph(self,self._checkpoint_name)
        
        trainable_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                                                scope="output")
        
        optimizer = AdamOptimizer(0.01, name="adam2")
        self._optimizer_op = optimizer.minimize(self._loss, var_list=trainable_variables)
        
        DNN_Classifier._restore_session(self, self._checkpoint_name)
        
        

In [ ]:
np.random.seed(40)
tf.random.set_random_seed(40)

transfer_classifier = DNN_Classifier_Transfer("Mnist-0to4-best_batch_norm")
transfer_classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)


### Crate DNN_Classifier_Frozen
- Same as DNN_Classifier_Transfer but it will pre calculate output of previous layer for all data

In [160]:
from imp import reload
import my_libs
reload(my_libs.dnn)
from my_libs.dnn import DNN_Classifier

class DNN_Classifier_Frozen(DNN_Classifier_Transfer):
        
    def _initialize_session_and_graph(self):
        DNN_Classifier_Transfer._initialize_session_and_graph(self)
        self._frozen_out = self._graph.get_tensor_by_name("DNN/hiden5_out:0")
        ## replace input placeholder _x with output of hidden layer _frozen_out
        self._old_x = self._x
        self._x = self._frozen_out
    
    
    def fit(self, x, y, val_x, val_y):
        self._initialize_session_and_graph()
        with self._session.as_default() as sess:
            self._x_frozen_out = self._frozen_out.eval(session=sess,feed_dict={self._old_x:x})
            self._val_x_frozen_out = self._frozen_out.eval(feed_dict={self._old_x:val_x})
            
        return DNN_Classifier.fit(self, self._x_frozen_out, y, self._val_x_frozen_out, val_y)
    

In [165]:
np.random.seed(40)
tf.random.set_random_seed(40)

frozen_classifier = DNN_Classifier_Frozen("Mnist-0to4-best_batch_norm")
frozen_classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)

epoch 0, score 0.501984, loss 1.376232
epoch 50, score 0.602891, loss 0.947225
epoch 100, score 0.603175, loss 1.030845
epoch 150, score 0.606576, loss 0.932684
epoch 200, score 0.602324, loss 0.827789
epoch 250, score 0.600340, loss 0.854857
No progress for 100 epoches.
Reverting back to epoch 189                     with 0.615646 score


DNN_Classifier_Frozen(checkpoint_name=None, reuse_layers=None)

### Test if running variable initializer resets previously set value

In [132]:
tf.reset_default_graph()
xx = tf.Variable(0, dtype=np.int16, name="xx")
inc = tf.assign(xx, xx+1)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(inc)
    sess.run(init)
    print(xx.eval())
    

0
